In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Intentar importar PyGIMLi
try:
    import pygimli as pg
    from pygimli.physics import VESManager
    print("✅ PyGIMLi disponible")
    PYGIMLI_AVAILABLE = True
except ImportError:
    print("⚠️ PyGIMLi no disponible")
    print("💡 Instalar con: conda install -c gimli pygimli")
    PYGIMLI_AVAILABLE = False

%matplotlib inline

## 🎯 ¿Qué es la Inversión Geofísica?

La **inversión** es el proceso de determinar las propiedades del subsuelo (resistividad, espesor de capas) a partir de las mediciones de campo (resistividad aparente).

### Problema directo vs inverso:

```
DIRECTO:  Modelo (capas) → Cálculo → Datos (resistividad aparente)
                ↓
INVERSO:  Datos (resistividad aparente) → Ajuste → Modelo (capas)
```

### Método de Occam (Navaja de Occam):
- Busca el modelo **más simple** que explique los datos
- Usa regularización para evitar modelos complejos
- Balance entre ajuste de datos y suavidad del modelo

---

## 📊 Preparar Datos de Ejemplo

In [ ]:
# Datos sintéticos de ejemplo (curva tipo H)
datos_sev = pd.DataFrame({
    'AB/2': [1.5, 2, 3, 4.5, 6, 9, 12, 18, 27, 40, 60, 90, 135, 200],
    'MN/2': [0.5, 0.5, 0.5, 0.5, 0.5, 1.5, 1.5, 1.5, 4.5, 4.5, 4.5, 13.5, 13.5, 13.5],
    'pa (Ω*m)': [45.2, 52.3, 68.5, 85.2, 95.8, 110.5, 125.3, 135.7, 140.2, 138.5, 132.8, 125.0, 115.3, 105.8]
})

print("📋 Datos de entrada para inversión:")
display(datos_sev)

# Visualizar curva
plt.figure(figsize=(10, 6))
plt.loglog(datos_sev['AB/2'], datos_sev['pa (Ω*m)'], 'o-', 
           markersize=8, linewidth=2, label='Datos observados')
plt.xlabel('AB/2 (m)', fontsize=12, fontweight='bold')
plt.ylabel('Resistividad aparente (Ω·m)', fontsize=12, fontweight='bold')
plt.title('Curva SEV - Datos de Entrada', fontsize=14, fontweight='bold')
plt.grid(True, which='both', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print("\n💡 Esta curva sugiere una estructura de 3 capas (tipo H):")
print("   - Capa superficial conductiva")
print("   - Capa intermedia resistiva")
print("   - Basamento más conductivo")

## ⚡ Inversión con PyGIMLi

### Parámetros clave:

- **nLayers**: Número de capas del modelo
- **lam (λ)**: Factor de regularización (suavidad del modelo)
  - λ pequeño: Más ajuste a los datos, modelo complejo
  - λ grande: Modelo más suave, menos ajuste
- **lambdaFactor**: Factor de reducción de λ por iteración

In [ ]:
if PYGIMLI_AVAILABLE:
    # Extraer datos
    ab2 = datos_sev['AB/2'].values
    mn2 = datos_sev['MN/2'].values
    rhoa = datos_sev['pa (Ω*m)'].values
    
    # Error estimado (3%)
    error = np.ones_like(rhoa) * 0.03
    
    # Crear VESManager
    ves = VESManager()
    
    print("✅ VESManager creado")
    print(f"📊 Datos: {len(ab2)} mediciones")
    print(f"📏 Rango AB/2: {ab2.min():.1f} - {ab2.max():.1f} m")
else:
    print("❌ PyGIMLi no disponible - No se puede ejecutar la inversión")
    print("💡 Instale PyGIMLi para continuar")

In [ ]:
if PYGIMLI_AVAILABLE:
    # Parámetros de inversión
    n_layers = 5
    lambda_val = 20
    lambda_factor = 0.8
    
    print("🔧 Parámetros de inversión:")
    print(f"   Número de capas: {n_layers}")
    print(f"   Lambda (λ): {lambda_val}")
    print(f"   Factor Lambda: {lambda_factor}")
    print("\n⏳ Ejecutando inversión...")
    
    # Ejecutar inversión
    model = ves.invert(
        rhoa, 
        error, 
        ab2=ab2, 
        mn2=mn2, 
        nLayers=n_layers,
        lam=lambda_val,
        lambdaFactor=lambda_factor
    )
    
    # Calcular chi-cuadrado (ajuste)
    chi2 = ves.inv.chi2()
    rms = np.sqrt(chi2)
    
    print("\n✅ Inversión completada")
    print(f"📊 Chi² = {chi2:.4f}")
    print(f"📊 RMS = {rms:.4f}")
    print(f"\n💡 RMS < 1 indica buen ajuste")

## 📊 Visualización de Resultados

In [ ]:
if PYGIMLI_AVAILABLE:
    # Extraer resultados
    thickness = model[:n_layers-1]  # Espesores
    resistivity = model[n_layers-1:]  # Resistividades
    depths = np.cumsum(thickness)  # Profundidades acumuladas
    
    # Crear figura con dos subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Subplot 1: Ajuste de datos
    ves.showData(rhoa, ab2=ab2, ax=ax1, label="Datos observados", 
                 color="C0", marker="o", markersize=8)
    ves.showData(ves.inv.response, ab2=ab2, ax=ax1, label="Curva ajustada", 
                 color="C1")
    ax1.set_xscale("log")
    ax1.set_yscale("log")
    ax1.set_xlabel("AB/2 (m)", fontsize=12, fontweight='bold')
    ax1.set_ylabel("Resistividad aparente (Ω·m)", fontsize=12, fontweight='bold')
    ax1.set_title(f"Ajuste del Modelo (RMS={rms:.4f})", fontsize=13, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, which='both', alpha=0.3)
    
    # Subplot 2: Modelo de capas
    max_depth = ab2.max() / 3  # Profundidad máxima de visualización
    ves.showModel(np.concatenate((thickness, resistivity)), ax=ax2, 
                  plot="semilogy", zmax=max_depth)
    ax2.set_title(f"Modelo de {n_layers} Capas", fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Gráficas generadas")

## 📋 Tabla de Resultados del Modelo

In [ ]:
if PYGIMLI_AVAILABLE:
    # Crear tabla de resultados
    resultados = pd.DataFrame({
        'Capa': range(1, n_layers + 1),
        'Espesor (m)': list(thickness) + ['∞'],
        'Profundidad (m)': list(depths) + ['∞'],
        'Resistividad (Ω·m)': [f"{r:.2f}" for r in resistivity]
    })
    
    print("📊 Modelo Invertido:")
    display(resultados)
    
    print("\n🔬 Interpretación geológica preliminar:")
    for i, (d, r) in enumerate(zip(list(depths) + [float('inf')], resistivity), 1):
        prof_texto = f"{d:.1f} m" if d != float('inf') else "∞"
        if r < 30:
            litologia = "Arcillas saturadas / Material muy conductivo"
        elif 30 <= r < 100:
            litologia = "Arcillas / Limos húmedos"
        elif 100 <= r < 300:
            litologia = "Arenas húmedas / Gravas con agua"
        elif 300 <= r < 1000:
            litologia = "Arenas secas / Gravas secas"
        else:
            litologia = "Roca consolidada / Material muy resistivo"
        
        print(f"   Capa {i} (hasta {prof_texto}): {r:.1f} Ω·m → {litologia}")

## 🔬 Análisis de Sensibilidad: Efecto de los Parámetros

Vamos a ver cómo diferentes valores de λ afectan el modelo:

In [ ]:
if PYGIMLI_AVAILABLE:
    # Probar diferentes valores de lambda
    lambdas = [5, 20, 50, 100]
    resultados_sensibilidad = []
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for idx, lam_val in enumerate(lambdas):
        print(f"⏳ Inversión con λ={lam_val}...")
        
        ves_temp = VESManager()
        model_temp = ves_temp.invert(
            rhoa, error, ab2=ab2, mn2=mn2, 
            nLayers=n_layers, lam=lam_val, lambdaFactor=0.8
        )
        
        chi2_temp = ves_temp.inv.chi2()
        rms_temp = np.sqrt(chi2_temp)
        
        resultados_sensibilidad.append({
            'lambda': lam_val,
            'chi2': chi2_temp,
            'rms': rms_temp
        })
        
        # Graficar
        ax = axes[idx]
        ves_temp.showData(rhoa, ab2=ab2, ax=ax, label="Observados", 
                         color="C0", marker="o", markersize=6)
        ves_temp.showData(ves_temp.inv.response, ab2=ab2, ax=ax, 
                         label="Ajustados", color="C1")
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_title(f"λ={lam_val}, RMS={rms_temp:.4f}", fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(True, which='both', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar tabla de comparación
    df_sensibilidad = pd.DataFrame(resultados_sensibilidad)
    print("\n📊 Comparación de parámetros:")
    display(df_sensibilidad)
    
    print("\n💡 Observaciones:")
    print("   - λ pequeño: Mejor ajuste (RMS menor) pero modelo más complejo")
    print("   - λ grande: Modelo más suave pero peor ajuste")
    print("   - λ óptimo: Balance entre ajuste y suavidad")

## 💾 Exportar Resultados

In [ ]:
if PYGIMLI_AVAILABLE:
    # Guardar modelo en Excel
    resultados.to_excel('modelo_invertido.xlsx', index=False)
    print("✅ Modelo guardado en 'modelo_invertido.xlsx'")
    
    # Crear reporte completo
    reporte = {
        'Parametros': {
            'Numero de capas': n_layers,
            'Lambda': lambda_val,
            'Factor Lambda': lambda_factor,
            'Puntos de datos': len(ab2)
        },
        'Calidad del ajuste': {
            'Chi cuadrado': chi2,
            'RMS': rms,
            'Rango AB/2': f"{ab2.min():.1f} - {ab2.max():.1f} m"
        }
    }
    
    print("\n📄 Reporte de inversión:")
    for seccion, datos in reporte.items():
        print(f"\n{seccion}:")
        for key, val in datos.items():
            print(f"   {key}: {val}")

## 🎯 Ejercicio Práctico

**Tarea**: 
1. Carga tus propios datos de campo
2. Realiza inversión con diferentes números de capas (3, 4, 5)
3. Compara los valores de RMS
4. Selecciona el modelo óptimo

```python
# Tu código aquí
```

## 📝 Resumen

En este tutorial aprendiste:

✅ **Concepto de inversión**: Pasar de datos a modelo  
✅ **Uso de PyGIMLi**: VESManager y parámetros  
✅ **Regularización**: Lambda (λ) y su efecto  
✅ **Evaluación**: Chi² y RMS  
✅ **Interpretación**: Correlación resistividad-litología  

### 🚀 Próximo paso

En el **Tutorial 4** aprenderás sobre:
- Generación de perfiles 2D
- Interpolación de múltiples SEV
- Visualización avanzada

---

**VESPY** - Vertical Electrical Sounding in Python  
📧 josemariagarciamarquez2.72@gmail.com  
🌐 [Web](https://josemariagarciamarquez.github.io/webjoma/) • ☕ [Patreon](https://www.patreon.com/chemitas)